In [1]:
import os
%pwd

'c:\\Dog_vs_Cat_Classifier\\trials'

In [2]:
os.chdir('../')

In [4]:
from main.d_vs_c.funtions.comon_funtn import read_yaml, create_directories
from main.d_vs_c.utils import *


class ConfigManeger:

    def __init__(self, config_path = config_file,
                 param_path= params_file):
        self.config = read_yaml(config_path)
        self.params = read_yaml(param_path)

        create_directories([self.config.artifacts_folder])

    def bilding_model_config(self):
        config = self.config.preproccessed_data
        create_directories([config.trained_model_path])

        # bild_model = (
        #     train_dir = Path(config.train_dir),
        #     test_dir= Path(config.test_dir),
        #     batch_size =self.params.batch_size,
        #     seed= self.params.seed
        # )
        bild_model = {
            'train_dir' : Path(config['train_dir']),
            'test_dir': Path(config['test_dir']),
            'batch_size': self.params['batch_size'],
            'seed': self.params['seed'],
            'image_size': self.params['image_size'],
            'conv_first_layer': self.params['conv_first_layer'],
            'conv_sec_layer': self.params['conv_sec_layer'],
            'conv_third_layer': self.params['conv_third_layer'],
            'conv_fourth_layer': self.params['conv_fourth_layer'],
            'input_shape': self.params['input_shape'],
            'kernel_size': self.params['kernel_size'],
            'pool_size': self.params['pool_size'],
            'strides': self.params['strides'],
            'dropout_rate': self.params['dropout_rate'],
            'dense_first_layer': self.params['dense_first_layer'],
            'dense_second_layer': self.params['dense_second_layer'],
            'output_layer': self.params['output_layer'],
            'l2_regularization': self.params['l2_regularization'],
            'optimizer': self.params['optimizer'],
            'loss': self.params['loss'],
            'metrics': self.params['metrics'],
            'epochs': self.params['epochs']
        }
        return bild_model

        

In [48]:
import keras
import tensorflow as tf

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.callbacks import EarlyStopping 
from keras import regularizers
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout


class Datasets:
    def __init__(self, config, params):
        self.config = config
        self.params = params

    def datasets_(self):
        train_ds = keras.utils.image_dataset_from_directory(
    directory = self.config['train_dir'],
    labels = 'inferred',
    label_mode = 'int',
    batch_size = self.params['batch_size'],
    seed = self.params['seed'],
    image_size = self.params['image_size']
)
        test_ds = keras.utils.image_dataset_from_directory(
    directory = self.config['test_dir'],
    labels = 'inferred',
    label_mode = 'int',
    batch_size = self.params['batch_size'],
    seed = self.params['seed'],
    image_size = self.params['image_size']
)
        return train_ds, test_ds
    

    # Normalixe the images
    @staticmethod
    def process(images, label):
        image = tf.image.resize(images, (256, 256))
        image = tf.image.convert_image_dtype(images, tf.float32)
        return images, label
    

    def model(self):
        model_3 = Sequential()
        
        model_3.add(Conv2D(self.params['conv_first_layer'], kernel_size=self.params['kernel_size'], padding='valid', activation='relu', input_shape=(self.params['input_shape'])))
        model_3.add(MaxPooling2D(pool_size=self.params['pool_size'], strides=self.params['strides'], padding='valid'))
        model_3.add(Dropout(self.params['dropout_rate']))
        
        model_3.add(Conv2D(self.params['conv_sec_layer'], kernel_size=self.params['kernel_size'], padding='valid', activation='relu'))
        model_3.add(MaxPooling2D(pool_size=self.params['pool_size'], strides=self.params['strides'], padding='valid'))
        model_3.add(Dropout(self.params['dropout_rate']))
        
        model_3.add(Conv2D(self.params['conv_third_layer'], kernel_size=self.params['kernel_size'], padding='valid', activation='relu'))
        model_3.add(MaxPooling2D(pool_size=self.params['pool_size'], strides=self.params['strides'], padding='valid'))
        model_3.add(Dropout(self.params['dropout_rate']))
        
        model_3.add(Conv2D(self.params['conv_fourth_layer'], kernel_size=self.params['kernel_size'], padding='valid', activation='relu'))
        model_3.add(MaxPooling2D(pool_size=self.params['pool_size'], strides=self.params['strides'], padding='valid'))
        model_3.add(Dropout(self.params['dropout_rate']))
        
        model_3.add(Flatten())
        
        model_3.add(Dense(self.params['dense_first_layer'], activation='relu', kernel_regularizer=regularizers.l2(self.params['l2_regularization'])))
        model_3.add(Dense(self.params['dense_second_layer'], activation='relu'))
        model_3.add(Dense(self.params['output_layer'], activation='sigmoid'))

         # Compile the model
        model_3.compile(optimizer=self.params['optimizer'], loss=self.params['loss'], metrics=self.params['metrics'])
        
        return model_3



    def run_model(self,model, train_ds, test_ds):
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        history = model.fit(train_ds, epochs=self.params['epochs'], validation_data=test_ds, callbacks=[early_stopping])
        return history



In [ ]:
import matplotlib.pyplot as plt
class plot_nd_save:
    def __init__(self, config, params):
        self.config = config
        self.params = params

    def plot_(self):


In [49]:


confi = ConfigManeger()
config = confi.bilding_model_config()
obj = Datasets(config, config)
train, test = obj.datasets_()
    
train_ds = train.map(obj.process)
test_ds = test.map(obj.process)
model = obj.model()
model.summary()
history = obj.run_model(model, train_ds, test_ds)


{'artifacts_folder': 'artifacts', 'downloading_data': {'source_dir': 'artifacts/raw_data', 'source_URL': 'karakaggle/kaggle-cat-vs-dog-dataset'}, 'preproccessed_data': {'datasets_dir': 'artifacts/datasets', 'train_dir': 'artifacts/datasets/train', 'test_dir': 'artifacts/datasets/test'}, 'image_categories': ['train/Dog', 'train/Cat', 'test/Dog', 'test/Cat']}
{'test_size': 0.2, 'random_state': 42, 'batch_size': 32, 'seed': 128, 'image_size': [256, 256], 'conv_first_layer': 32, 'conv_sec_layer': 64, 'conv_third_layer': 128, 'conv_fourth_layer': 256, 'kernel_size': [3, 3], 'input_shape': [256, 256, 3], 'pool_size': [2, 2], 'strides': 2, 'dropout_rate': 0.25, 'dense_first_layer': 128, 'dense_second_layer': 64, 'output_layer': 1, 'l2_regularization': 0.01, 'optimizer': 'adam', 'loss': 'binary_crossentropy', 'metrics': ['accuracy'], 'epochs': 50}
[2024-11-20 00:19:39,357: INFO: comon_funtn: created directory at: artifacts]


Found 19963 files belonging to 2 classes.
Found 4986 files belonging to 2 classes.


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_33 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_34 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_35 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 128)            │     6,422,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,819,393 (26.01 MB)

 Trainable params: 6,819,393 (26.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
  4/624 ━━━━━━━━━━━━━━━━━━━━ 22:24 2s/step - accuracy: 0.5104 - loss: 529.7144

: 